In [ ]:
# To be done only once
#!cythonize Analyze_cython.pyx -i
#!cythonize sbl_wrapper.pyx -i

In [ ]:
from pathlib import Path
import numpy as np
from DLCA_Reader import DLCA

# The folder with all .h5 and .xmf files
data_dir = Path("output_dir/")

# Read all data
Spheres, Aggregates = DLCA(data_dir).read()

In [ ]:
# Create a new collumn in Aggregates that contains the giration diameter of all aggregates
Aggregates["Dg"] = 2 * Aggregates["Rg"]
Spheres["Diameter"] = 2 * Spheres["Radius"]

# Create a new collumn in Aggregates that contains the mean diameter the spheres of each aggregates
# This also means that we need to compute
# the mean of the diameter of all the spheres that shares the same time and the same label
# Reminder : label is the unique number given to an aggregate to recognize it
Aggregates["MeanDp"] = Spheres.groupby(by=["Time", "Label"])["Diameter"].mean()

# Compute Dg over Dp
Aggregates["DgOverDp"] = Aggregates["Dg"] / Aggregates["MeanDp"]

In [ ]:
#select aggregates from the first half time period
times = Aggregates.index.get_level_values(0)

tlimit = times.max() / 2
Selection = Aggregates.loc[:tlimit]

#select aggregates with less than 50 spheres
Selection = Selection[Selection.Np <50]

# get the aggregates with the largest ratio Dg / Dp
t, label = Selection["DgOverDp"].idxmax()
Agg_max_Rg = Selection.loc[t, label]

print(Agg_max_Rg)
print("The name here is (time, label)")

In [ ]:
# get the coresponding spheres
Sph_max_Rg = Spheres[Spheres["Label"] == label].loc[t]
print(Sph_max_Rg)

#write to file
Sph_max_Rg.to_csv("MyAggregate.csv", header=True)

In [ ]:
%matplotlib notebook
from DLCAtools import view_proj_agg

view_proj_agg(Agg_max_Rg, Spheres)
view_proj_agg(Agg_max_Rg, Spheres, reduce=np.max)

In [ ]:
%matplotlib notebook
from DLCAtools import view_agg
    
Agg = Agg_max_Rg
#Agg = Aggregates.loc[0., 0.]
view_agg(Agg, Spheres)

In [ ]:
#Compute the mean for each time step
time_agg = Aggregates.groupby(by="Time").agg((np.mean))

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.loglog(time_agg["Np"])
ax.set_xlabel('Time (s)', fontsize=9)
ax.set_ylabel('Mean Np', fontsize=9)
fig.suptitle('Time evolution of Np', fontsize=11)
plt.show()


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.loglog((time_agg["Dg"]))
ax.set_xlabel('Time (s)', fontsize=9)
ax.set_ylabel('Dg moyen (nm)', fontsize=9)
plt.suptitle('Variation du Dg dans le temps', fontsize=11)
plt.show()


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn import linear_model

fig, ax = plt.subplots()
ax.loglog(Aggregates["DgOverDp"],
        Aggregates["Np"],'.', label="Aggregates")
ax.set_xlabel('Dg/Dp', fontsize=9)
ax.set_ylabel('Mean Np', fontsize=9)

# ignore the too small aggregates
subset = Aggregates[Aggregates.Np > 2]

# fit the fractal law (not the simplest way to do it)
model = linear_model.LinearRegression()
model.fit(np.log(subset["DgOverDp"])[:, np.newaxis],
          np.log(subset["Np"]))

print("Fractal Law: ", np.exp(model.intercept_), "x^", *model.coef_)

# Predict data of estimated models
X = np.linspace(np.log(subset["DgOverDp"]).min(),
                np.log(subset["DgOverDp"]).max())[:, np.newaxis]
Y = model.predict(X)
X = np.exp(X)
Y = np.exp(Y)

plt.plot(X,Y, label="Fit")
fig.suptitle('Evolution of Np in function of Dg/Dp', fontsize=11)
plt.legend()
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from DLCAtools import coverages

# Compute the overlapping coefficient for all the aggregates
Aggregates["cov"] = coverages(Spheres, Aggregates)

#Filter out aggregates that have only one spheres
cov = Aggregates["cov"][Aggregates["Np"] > 1]

#Compute the mean overlapping coefficient for each time step
time_cov = cov.groupby(by="Time").agg((np.mean))

#Plot the evolution of the overlapping coefficient in time.
fig, ax = plt.subplots()
ax.loglog(time_cov)
ax.set_xlabel('Time (s)', fontsize=9)
ax.set_ylabel('Overlapping coefficient', fontsize=9)
plt.suptitle('Time evolution of the overlapping coefficient', fontsize=11)
plt.show()


In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def graph(bins,debut, fin, biny=0):
    
    # get the data
    data = Aggregates["Dg"].loc[debut:fin, :]
    
    # compute the histogram
    hist, binx = np.histogram(data, bins=bins)
    
    # bar3d does not work in logscale
    binx = np.log10(binx)

    # normalize all the histograms (we may have more or less time step for each time range)
    hist_norm = hist / hist.sum()
    
    # plot the histogram
    surf = ax.bar3d(binx[:-1], biny, 0,
                     dx = np.diff(binx), dy = 0.5, dz = hist_norm,
                    alpha=0.5, label="{:5.2e}s -> {:5.2e}s".format(debut, fin))

    # workaround an another matplotlib bug
    surf._facecolors3d = mpl.colors.to_rgba_array(
                          surf._facecolors3d, surf._alpha)
    surf._edgecolors3d = mpl.colors.to_rgba_array(
                          surf._edgecolors3d, surf._alpha)
    surf._facecolors2d=surf._facecolors3d
    surf._edgecolors2d=surf._edgecolors3d
    

# Split the time range in even parts
times = Aggregates.index.get_level_values(0)
limits = np.linspace(0, times.max()/5, 5)

# Create the categories
bins = np.logspace(np.log10(Aggregates["Dg"].min()),
                   np.log10(Aggregates["Dg"].max()), 20)

# Create a new figure (we need it to put all graphs on the same figure)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i,(debut, fin) in enumerate(zip(limits[:-1], limits[1:])):
    graph(bins, debut, fin, i)
        
ax.set_xlabel('log10(Dg)')
plt.suptitle("Histogram of the gyration diameter for some period of time")
ax.legend()
plt.show()

In [ ]:
from DLCAtools import inertia

Agg = Agg_max_Rg
#Agg = Aggregates.loc[0., 0.]

inertia_agg = inertia(Agg, Spheres)
print("Gyration radius (code): ", Agg.Rg)
print("Gyration radius:        ", np.sqrt(np.sum(inertia_agg)/2))
print("Anisotropy:             ", inertia_agg.max()/inertia_agg.min())


In [ ]:
from DLCAtools import surface_volume_sbl
Agg = Aggregates.loc[0., 0]

print("Sphere")
print("  DLCA")
print("    Volume:  ", Agg.Volume)
print("    Surface: ", Agg.Surface)
print()
print("  SBL")
volsbl, surfsbl = surface_volume_sbl(Agg, Spheres)
print("    Volume:  ", volsbl)
print("    Surface: ", surfsbl)

Agg = Agg_max_Rg
print()
print("Aggregate rg max")
print("  DLCA")
print("    Volume:  ", Agg.Volume)
print("    Surface: ", Agg.Surface)
print()
print("  SBL")
volsbl, surfsbl = surface_volume_sbl(Agg, Spheres)
print("    Volume:  ", volsbl)
print("    Surface: ", surfsbl)

Agg = Aggregates.iloc[-1]
print()
print("Last Aggregate")
print("  DLCA")
print("    Volume:  ", Agg.Volume)
print("    Surface: ", Agg.Surface)
print()
print("  SBL")
volsbl, surfsbl = surface_volume_sbl(Agg, Spheres)
print("    Volume:  ", volsbl)
print("    Surface: ", surfsbl)


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from DLCAtools import autocorrelation3d

Agg = Aggregates.iloc[-1]
radius, volumes = autocorrelation3d(Agg, Spheres, 32, 16)

plt.figure()
plt.loglog(radius, volumes, '-o')
plt.show()

In [ ]:
import numpy as np
p = (np.log(0.445) - np.log(0.00309))/(np.log(1.29e-8) - np.log(7.16e-7))
#p = (np.log(0.000674) - np.log(0.00234))/(np.log(4.73e-7) - np.log(1.35e-7))
print(3 + p)

In [ ]:
from DLCAtools import export_ddscat

Agg = Aggregates.iloc[-1]

export_ddscat(Agg, Spheres, type_limits=[-0.2, 0.])

**Have Fun !**